In [1]:
# # 📦 Importar librerías necesarias
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, count, isnan, when, trim, upper, lower, expr
from pyspark.sql.types import IntegerType

## GUIA DE COMANDOS SPARK PARA LA LIMPIEZA

In [5]:
# # 🚀 Crear sesión Spark
# spark = SparkSession.builder.appName("LimpiezaDatos").getOrCreate()

# # 📥 Cargar datos desde CSV
# df = spark.read.csv("archivo.csv", header=True, inferSchema=True)

# # 👁️ Visualizar primeras filas y esquema
# df.show(5)                 # Primeras 5 filas
# df.printSchema()           # Tipos de columnas
# df.columns                 # Lista de nombres de columnas

# # 📊 Resumen estadístico
# df.describe().show()       # count, mean, stddev, min, max
# df.summary().show()        # count, mean, stddev, min, 25%, 50%, 75%, max

# # 🧼 Limpieza de datos

# ## 1. Verificar nulos
# df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()

# ## 2. Eliminar filas con nulos
# df = df.dropna()

# ## 3. Rellenar nulos con valores por defecto
# df = df.fillna({
#     "columna_numerica": 0,
#     "columna_categorica": "desconocido"
# })

# ## 4. Reemplazar valores nulos en una columna
# df = df.withColumn("columna", when(col("columna").isNull(), "valor_defecto").otherwise(col("columna")))

# ## 5. Eliminar duplicados
# df = df.dropDuplicates()  # O: df.dropDuplicates(["columna_clave"])

# ## 6. Renombrar columnas
# df = df.withColumnRenamed("Col Antiguo", "col_nuevo")

# ## 7. Conversión de tipos
# df = df.withColumn("edad", col("edad").cast("int"))
# df = df.withColumn("fecha", col("fecha").cast("date"))

# ## 8. Limpiar y modificar texto
# df = df.withColumn("col_trim", trim(col("col_texto")))
# df = df.withColumn("col_upper", upper(col("col_texto")))
# df = df.withColumn("col_lower", lower(col("col_texto")))

# ## 9. Crear o modificar columnas
# df = df.withColumn("suma", col("col1") + col("col2"))
# df = df.withColumn("categoria_edad", when(col("edad") >= 18, "adulto").otherwise("menor"))

# ## 10. Filtrar datos
# df = df.filter(col("ingresos") > 1000)
# df = df.filter(col("pais") == "España")

# ## 11. Eliminar columnas innecesarias
# df = df.drop("columna_innecesaria")  # Una
# df = df.drop(*["col1", "col2"])      # Varias

# ## 12. Reemplazar valores
# df = df.replace("desconocido", "otros", subset=["columna"])

# ## 13. Ver valores únicos y categorías
# df.select("columna").distinct().show()
# df.groupBy("categoria").count().show()

# # 📈 Visualización básica (limitada en Spark)
# df.show(10)                       # Muestra 10 filas
# df.groupBy("columna").count().show()  # Conteo por categoría

# # Si quieres graficar: convertir a Pandas (⚠️ cuidado con datasets grandes)
# df_pd = df.limit(1000).toPandas()

# # Ejemplo: usar matplotlib o seaborn
# import matplotlib.pyplot as plt
# import seaborn as sns

# sns.histplot(df_pd["edad"], bins=20)
# plt.title("Distribución de edad")
# plt.show()

# sns.countplot(data=df_pd, x="categoria")
# plt.title("Conteo por categoría")
# plt.xticks(rotation=45)
# plt.show()


In [2]:
spark = SparkSession.builder.appName('tfm_2025').getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/08 12:33:48 WARN Utils: Your hostname, DESKTOP-JCU85UN, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/08 12:33:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/08 12:33:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Primer archivo "EloRatings.csv"

In [3]:
# Cargar el primer csv
df = spark.read.csv("../Data/EloRatings.csv", header=True, inferSchema=True)

# Ver las primeras filas
df.show(10)
df.printSchema()
df.columns

+----------+---------+-------+-------+
|      date|     club|country|    elo|
+----------+---------+-------+-------+
|2000-07-01|   Aachen|    GER| 1453.6|
|2000-07-01|  Aalborg|    DEN|1482.61|
|2000-07-01|    Aalst|    BEL|1337.53|
|2000-07-01|   Aarhus|    DEN|1381.46|
|2000-07-01| Aberdeen|    SCO|1360.43|
|2000-07-01|Adanaspor|    TUR|1380.76|
|2000-07-01|      AEK|    GRE|1599.31|
|2000-07-01|      AIK|    SWE|1563.87|
|2000-07-01|  Ajaccio|    FRA|1470.87|
|2000-07-01|     Ajax|    NED|1604.75|
+----------+---------+-------+-------+
only showing top 10 rows
root
 |-- date: date (nullable = true)
 |-- club: string (nullable = true)
 |-- country: string (nullable = true)
 |-- elo: double (nullable = true)



['date', 'club', 'country', 'elo']

In [ ]:
def mostrar_sumario(df):
    """
    Muestra un sumario del dataset.
    Explicacion:
        - Usa la funcion 'describe()' para mostrar las siguientes metricas:
            - El conteo de filas por columna.
            - La media de los valores de las columnas numericas (muestra NULL si no es numerica).
            - La desviacion  estandar de los valores de las columnas numericas (muestra NULL si no es numerica).
            - El valor minimo de cada columna.
            - El valor maximo de cada columna.
        - Devuelve el DataFrame con las metricas.
    Args:
        df: DataFrame a analizar

    Returns:
        Un DataFrame con las métricas del dataset.
    """
    return df.summary()

mostrar_sumario(df).show()

+-------+------------+-------+------------------+
|summary|        club|country|               elo|
+-------+------------+-------+------------------+
|  count|      242591| 242591|            242591|
|   mean|        NULL|   NULL|1505.1128453652552|
| stddev|        NULL|   NULL| 155.5037772431839|
|    min|A. Sebatspor|    AUT|           1069.68|
|    25%|        NULL|   NULL|           1394.83|
|    50%|        NULL|   NULL|           1486.94|
|    75%|        NULL|   NULL|           1606.08|
|    max|      Zwolle|    TUR|           2107.48|
+-------+------------+-------+------------------+



## CONTAMOS LOS NULOS QUE HAY EN CADA COLUMNA

In [ ]:
def contar_nulos_por_columna(df):
    """
    Calcula el numero de valores nulos en cada columna del DataFrame.
    Explicacion:
        - Itera sobre cada columna del DataFrame utilizando 'df.columns'.
        - Usa la funcion 'expr()' que utiliza una expresion SQL para contar los valores nulos en cada columna.
        - Asigna '1' si el valor es nulo y '0' si no es nulo.
        - Renombra la columna con '.alias(c)' para mantener el nombre original.
        - Selecciona y aplica la transformacion en todas las columnas dinamicamente con 'df.select([...])'.
        - Devuelve el DataFrame con los conteos de nulos.
    Args:
        df: DataFrame a analizar

    Returns:
        Un DataFrame con las mismas columnas del DataFrame original
        y una fila que indica el numero de valores nulos por columna.
    """
    # return df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
    return df.select([expr(f"sum(case when {c} is null then 1 else 0 end)").alias(c) for c in df.columns])

contar_nulos_por_columna(df).show()


+----+----+-------+---+
|date|club|country|elo|
+----+----+-------+---+
|   0|   0|      0|  0|
+----+----+-------+---+



In [ ]:
# COMPROBAR SI HAY VALORES DUPLICADOS
df.exceptAll(df.dropDuplicates()).show()

+----+----+-------+---+
|date|club|country|elo|
+----+----+-------+---+
+----+----+-------+---+



In [ ]:
# Mostrar nombres unicos de clubs
pd.Series(df.values.flatten()).unique()

In [ ]:
df = df.withColumn("gross", regexp_replace(col("gross"), "[$,M]", "")) # Eliminamo $ y M.
df = df.withColumn("gross", (col("gross").cast("double") * 1000000).cast("int")) # Convertimos a float, multiplicamos por 1 millon, y lo convertimos a integer.
df = df.withColumnRenamed("gross", "gross(M)") # Cambiamos el nombre de la columna.

In [ ]:
# Ruta donde quieres guardar el archivo CSV en Google Drive
output_path = "../Data/EloRatings_cleaned"

# Guardar el DataFrame en formato CSV
# Spark por defecto guarda en particiones
# Para guardarlo en un solo archivo, usar repartition(1) antes de 'write'
df.repartition(1).write.csv(output_path, header=True, mode="overwrite")

# Segundo archivo "Matches.csv"

In [12]:
spark.stop()